In [1]:
import pandas as pd
import json 
import numpy as np 
import sqlite3

In [2]:
#Connect to the database
con = sqlite3.connect('..\database\sqlite-tools-win32-x86-3380300\database.db')

In [3]:
#Query to pull thing down 
query_gen = '''
    SELECT *
    FROM general_tweets 
'''
df_data_gen = pd.read_sql_query(query_gen, con)

query_klm = '''
    SELECT *
    FROM KLM_tweets 
'''
df_data_klm = pd.read_sql_query(query_klm, con)

query_bri = '''
    SELECT *
    FROM british_airways_tweets 
'''
df_data_bri = pd.read_sql_query(query_bri, con)

In [4]:
#This database thing is fuking stupid. We need another solution
df_data_bri.set_index('', inplace=True)
df_data_gen.set_index('', inplace=True)
df_data_klm.set_index('', inplace=True);

In [5]:
#Drop some NA, I do remember I already did this before upload to database, but double check
df_data_drop_na = df_data_gen.dropna(subset=['user'])
len(df_data_gen), len(df_data_drop_na)

(134166, 134166)

In [11]:
#This code I got from Xiaoya with some editing 
def remove_duplicated_text(df_data: pd.DataFrame) -> pd.DataFrame:
    """
    Function to remove duplicated datapoints that send by users but not airlines.
    Tweets are considered to be a duplicate if they are sent by the same user to the same tweet. 
    Drop completely-duplicated datapoints
    This function resets index.
    
    parameter: df_data: DataFrame
    return: DataFrame
    """
    #remove duplicated tweets sent by users not KLM or Bri
    df_tweets = df_data.reset_index(drop=True)
    df_tweets = df_tweets.dropna(subset=['user'])
    df_tweets.drop_duplicates(inplace=True)
    du = df_tweets.duplicated(subset=['text', 'user_id', 'in_reply_to_status_id'])
    #get a list index of duplicated text (datapoint)
    lst = [index for index, values in du.items() if values == True]
    #define airline dictionary
    airline = {
        'KLM' : 56377143, 
        'AirFrance' : 106062176, 
        'British_Airways' : 18332190, 
        'AmericanAir' : 22536055, 
        'Lufthansa' : 124476322, 
        'AirBerlin' : 26223583, 
        'AirBerlin assist' : 2182373406, 
        'easyJet' : 38676903, 
        'RyanAir' : 1542862735, 
        'SingaporeAir' : 253340062, 
        'Qantas' : 218730857, 
        'EtihadAirways' : 45621423, 
        'VirginAtlantic' : 20626359 
    }
    #drop the duplicated texts that not from any airlines
    drop_lst = []
    for i in lst:
        if df_tweets.loc[i, 'user_id'] not in airline.values():
            drop_lst.append(i)
    df_tweets.drop(index=drop_lst, inplace=True)
    return df_tweets

In [12]:
#Use the function to remove duplicated
df_clean_klm = remove_duplicated_text(df_data_klm)
df_clean_bri = remove_duplicated_text(df_data_bri)
df_clean_gen = remove_duplicated_text(df_data_gen)

In [18]:
len(df_clean_gen), len(df_data_gen)

(134166, 134166)

In [44]:
#Turn to csv file to upload on database
df_clean_klm.to_csv('../database/CSV file/clean_klm_table.csv')
df_clean_bri.to_csv('../database/CSV file/clean_british_airways_table.csv')
df_clean_gen.to_csv('../database/CSV file/clean_general_table.csv')